In [15]:
import requests
from full_obj import myobj

Testando o v1

In [16]:
url = 'http://127.0.0.1:8000/v1/'
#myobj = [
#     {
#        "VAR2": "M",
#        "IDADE": 43.893,
#        "VAR5": "PR",
#        "VAR6": -25.4955709,
#        "VAR7": -49.2454987,
#        "VAR8": "D",
#        "VAR9": "E",
#        "VAR10": "MEDIA",
#        "VAR11": 1.0,
#        "VAR12": 0.182,
#        "VAR14": 0.597,
#        "VAR15": 0.618,
#        "VAR16": 0.25,
#        "VAR18": 1.076712,
#        "VAR19": 5.057534,
#        "VAR22": 0.125,
#        "VAR24": 0.069,
#        "VAR25": 0.0969999999999999,
#        "VAR32": "SALDO INEXISTENTE",
#        "VAR39": 0.661039,
#        "VAR40": 0.573539,
#        "VAR41": 0.4793699999999999,
#        "VAR42": 0.4440489999999999,
#        "VAR47": 0.006,
#        "VAR49": "S",
#        "VAR50": "N",
#        "VAR51": "N",
#        "VAR52": "N",
#        "VAR53": "N",
#        "VAR54": "N",
#        "VAR55": "N",
#        "VAR56": "S",
#        "VAR57": "S",
#        "VAR58": "N",
#        "VAR59": "N",
#        "VAR60": "N",
#        "VAR61": "N",
#        "VAR62": "N",
#        "VAR63": "N",
#        "VAR64": "N",
#        "VAR65": "N",
#        "VAR66": "ALTISSIMA",
#        "VAR67": "ALTA",
#        "VAR68": "ALTISSIMA",
#        "VAR69": "ALTISSIMA",
#        "VAR70": "ALTISSIMA",
#        "VAR71": "ALTA",
#        "VAR72": "ALTISSIMA",
#        "VAR73": "ALTISSIMA",
#        "VAR74": "ALTISSIMA",
#        "VAR75": "ALTISSIMA",
#        "VAR76": "ALTA",
#        "VAR77": "ALTISSIMA",
#        "VAR78": "ALTISSIMA",
#        "VAR79": "ALTISSIMA",
#        "VAR80": "ALTA",
#        "VAR81": "ALTA",
#        "VAR82": "ALTISSIMA",
#        "VAR83": "ALTISSIMA",
#        "VAR84": "ALTA",
#        "VAR85": "ALTA",
#        "VAR86": "ALTA",
#        "VAR87": "ALTISSIMA",
#        "VAR88": "ALTA",
#        "VAR89": "ALTISSIMA",
#        "VAR90": "BAIXISSIMA",
#        "VAR91": "ALTA",
#        "VAR92": "ALTISSIMA",
#        "VAR93": "ALTISSIMA",
#        "VAR94": "ALTISSIMA",
#        "VAR95": "ALTA",
#        "VAR96": "ALTISSIMA",
#        "VAR97": "ALTA",
#        "VAR98": "ALTISSIMA",
#        "VAR99": "ALTISSIMA",
#        "VAR100": "BAIXISSIMA",
#        "VAR101": "ALTA",
#        "VAR102": "MEDIO",
#        "VAR103": "MEDIO",
#        "VAR104": "PROXIMO",
#        "VAR105": "LONGE",
#        "VAR106": "MEDIO",
#        "VAR107": "MEDIO",
#        "VAR108": "MEDIO",
#        "VAR109": "MEDIO",
#        "VAR110": "PROXIMO",
#        "VAR111": "MEDIO",
#        "VAR112": "MEDIO",
#        "VAR113": "PROXIMO",
#        "VAR114": "PROXIMO",
#        "VAR115": "MEDIO",
#        "VAR116": "LONGE",
#        "VAR117": "MEDIO",
#        "VAR118": "MEDIO",
#        "VAR119": "LONGE",
#        "VAR120": "MUITO LONGE",
#        "VAR121": "MEDIO",
#        "VAR122": "MEDIO",
#        "VAR123": "MEDIO",
#        "VAR124": "MEDIO",
#        "VAR125": "PROXIMO",
#        "VAR126": "MEDIO",
#        "VAR127": "PROXIMO",
#        "VAR128": "LONGE",
#        "VAR129": "MEDIO",
#        "VAR130": "MEDIO",
#        "VAR131": "MEDIO",
#        "VAR132": "PROXIMO",
#        "VAR133": "MEDIO",
#        "VAR134": "PROXIMO",
#        "VAR135": "MEDIO",
#        "VAR136": "MEDIO",
#        "VAR137": "MEDIO",
#        "VAR138": "MEDIO",
#        "VAR139": "MEDIO",
#        "VAR140": "MUITO PROXIMO",
#        "VAR141": 3970.113648,
#        "VAR142": "C",
#        "REF_DATE": "2017-03-25 00:00:00+00:00",
#        "TARGET": 1
#    },
#    {
#        "VAR2": "F",
#        "IDADE": 31.559,
#        "VAR5": "RS",
#        "VAR6": -30.1090924,
#        "VAR7": -51.3238273,
#        "VAR8": "E",
#        "VAR9": "E",
#        "VAR10": "ALTISSIMA",
#        "VAR11": 0.0,
#        "VAR12": 0.055,
#        "VAR14": 0.22,
#        "VAR15": 0.191,
#        "VAR18": 0.690411,
#        "VAR19": 2.189041,
#        "VAR22": 0.375,
#        "VAR24": 0.4479999999999999,
#        "VAR25": 0.065,
#        "VAR32": "DECLARACAO NAO ENVIADA",
#        "VAR39": 0.429179,
#        "VAR40": 0.731167,
#        "VAR41": 0.75676,
#        "VAR42": 0.452576,
#        "VAR47": 0.001,
#        "VAR49": "S",
#        "VAR50": "N",
#        "VAR51": "N",
#        "VAR52": "N",
#        "VAR53": "N",
#        "VAR54": "N",
#        "VAR55": "N",
#        "VAR56": "N",
#        "VAR57": "N",
#        "VAR58": "N",
#        "VAR59": "N",
#        "VAR60": "N",
#        "VAR61": "N",
#        "VAR62": "N",
#        "VAR63": "N",
#        "VAR64": "N",
#        "VAR65": "N",
#        "VAR66": "BAIXISSIMA",
#        "VAR67": "BAIXISSIMA",
#        "VAR68": "BAIXISSIMA",
#        "VAR69": "ALTISSIMA",
#        "VAR70": "BAIXISSIMA",
#        "VAR71": "MEDIA",
#        "VAR72": "BAIXISSIMA",
#        "VAR73": "BAIXISSIMA",
#        "VAR74": "BAIXISSIMA",
#        "VAR75": "MEDIA",
#        "VAR76": "MEDIA",
#        "VAR77": "BAIXISSIMA",
#        "VAR78": "BAIXISSIMA",
#        "VAR79": "BAIXISSIMA",
#        "VAR80": "BAIXISSIMA",
#        "VAR81": "MEDIA",
#        "VAR82": "BAIXISSIMA",
#        "VAR83": "BAIXISSIMA",
#        "VAR84": "BAIXISSIMA",
#        "VAR85": "BAIXISSIMA",
#        "VAR86": "BAIXISSIMA",
#        "VAR87": "BAIXISSIMA",
#        "VAR88": "BAIXA",
#        "VAR89": "BAIXISSIMA",
#        "VAR90": "BAIXISSIMA",
#        "VAR91": "MEDIA",
#        "VAR92": "BAIXISSIMA",
#        "VAR93": "BAIXISSIMA",
#        "VAR94": "BAIXISSIMA",
#        "VAR95": "BAIXISSIMA",
#        "VAR96": "BAIXA",
#        "VAR97": "MEDIA",
#        "VAR98": "BAIXISSIMA",
#        "VAR99": "MEDIA",
#        "VAR100": "BAIXISSIMA",
#        "VAR101": "BAIXISSIMA",
#        "VAR102": "LONGE",
#        "VAR103": "MUITO LONGE",
#        "VAR104": "MUITO LONGE",
#        "VAR105": "LONGE",
#        "VAR106": "MUITO LONGE",
#        "VAR107": "LONGE",
#        "VAR108": "MEDIO",
#        "VAR109": "LONGE",
#        "VAR110": "LONGE",
#        "VAR111": "MUITO LONGE",
#        "VAR112": "LONGE",
#        "VAR113": "LONGE",
#        "VAR114": "LONGE",
#        "VAR115": "LONGE",
#        "VAR116": "MUITO PROXIMO",
#        "VAR117": "LONGE",
#        "VAR118": "LONGE",
#        "VAR119": "PROXIMO",
#        "VAR120": "MUITO LONGE",
#        "VAR121": "PROXIMO",
#        "VAR122": "MUITO LONGE",
#        "VAR123": "MUITO LONGE",
#        "VAR124": "MUITO LONGE",
#        "VAR125": "LONGE",
#        "VAR126": "LONGE",
#        "VAR127": "MEDIO",
#        "VAR128": "MUITO PROXIMO",
#        "VAR129": "MUITO LONGE",
#        "VAR130": "LONGE",
#        "VAR131": "LONGE",
#        "VAR132": "MUITO PROXIMO",
#        "VAR133": "MUITO PROXIMO",
#        "VAR134": "MUITO LONGE",
#        "VAR135": "MUITO PROXIMO",
#        "VAR136": "MEDIO",
#        "VAR137": "LONGE",
#        "VAR138": "LONGE",
#        "VAR139": "LONGE",
#        "VAR140": "MUITO PROXIMO",
#        "VAR141": 1513.2274,
#        "VAR142": "E",
#        "REF_DATE": "2017-08-31 00:00:00+00:00",
#        "TARGET": 1
#    }
#]

In [17]:
x = requests.post(url,json=myobj)

In [18]:
x.text

'{"volumetria":{"July":74,"August":72,"May":67,"June":63,"March":62,"January":58,"February":55,"April":49},"ROC-AUC":0.5751748251748252}'

Testando o dataset oot

In [19]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Foot.gz'

In [20]:
x = requests.post(url)


In [21]:
print(x.text)

{"estatistica de Kolmogorov-Smirnov":0.09172708533663676,"p":1.0236104206010673e-225}


Testando o dataset de teste

In [22]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Ftest.gz'

In [23]:
x = requests.post(url)

In [24]:
print(x.text)

{"estatistica de Kolmogorov-Smirnov":0.0,"p":1.0}


Testando a estatistica de Mannwithneyu

In [25]:
url = 'http://127.0.0.1:8000/v2/?file_path=..%2F..%2Fdatasets%2Fcredit_01%2Foot.gz&test=MAN'


In [26]:
x = requests.post(url)

In [27]:
print(x.text)

{"estatistica de mannwithneyu":1859523611.5,"p":1.7244757437036297e-111}
